# Example queries 

this notebook compares the diffrent datasources based on the queries defined in the example_queries.md

### Parameters and credentials

the code cell below defines the list of analyzed hashtags and the time range to be compared.
IMPORTANT do not forget to set the credentials for the individual databases.

In [1]:
# Hashtags should be case insensitive. This means that when you are adding `#missingmaps` or `#MissingMaps` or `#missingMaps` to a changeset comment this will all be counted towards lower case `missingmaps`. This is currently the case for the processing of the osmstats-api.

hashtags = ['missingmaps','MissingMaps', 'bloomberg','visa','hotosm-project-14154','hotosm-project-*']


timerange = ["2015-01-01","2022-12-31"]

# credentials for the OSM-stats API
creds_mm = {"u":"user@host",
            "h": "host",
            "port": "port",
            "db": "db_name",
            "pw": "password"}

# credentials for the OSM changeset DB
creds_chDB =  {"u":"postgres",
            "h": "localhost",
            "port": "5432",
            "db": "postgres",
            "pw": "Eichen12"}

# credentials for the OSHDB Tables 
creds_OSHDB =  {"u":"user@host",
            "h": "host",
            "port": "port",
            "db": "db_name",
            "pw": "password"}


In [3]:
import psycopg2 as ps 
from psycopg2._psycopg import connection
from pandas import DataFrame

def osmStatsQuery(timerange:[str],hashtag:str,conn:connection,aggregator:str="COUNT(DISTINCT user_id)"):
    query = f"""SELECT {aggregator} FROM raw_changesets as raw WHERE (created_at BETWEEN '{timerange[0]}' AND '{timerange[1]}') AND raw.id in 
                (SELECT changeset_id FROM raw_changesets_hashtags as hsh where hsh.hashtag_id in 
                (SELECT id FROM raw_hashtags as rh WHERE LOWER(rh.hashtag) LIKE LOWER('{hashtag}')))""" 
    with conn.cursor() as cur:
        cur.execute(query)
        res = cur.fetchall()
    return res

def changesetDbQuery(timerange:[str],hashtag:str,conn:connection,aggregator:str="COUNT(DISTINCT user_id)"):
    query = f"""SELECT {aggregator} 
                FROM osm_changeset
                WHERE (LOWER(tags -> 'comment') LIKE '%#visa%' or LOWER(tags -> 'hashtags') LIKE LOWER('%#visa%'))      
                AND (created_at BETWEEN '{timerange[0]}' AND '{timerange[1]}');
                """
    with conn.cursor() as cur:
        cur.execute(query)
        res = cur.fetchall()
    return res

"""
def parquetQuery(timerange:[str],hashtag:str,filepath:str,coumns):->DataFrame
    #toDO
    return"""

'\ndef parquetQuery(timerange:[str],hashtag:str,filepath:str,coumns):->DataFrame\n    #toDO\n    return'

## EQ1 Number of OSM Contributors

This is the number of distinct OSM users that contributed at least one changeset to a particular hashtag since the start of collecting data. This is the very first number displayed on the company leaderboards as `contributors`.

In [7]:
#with ps.connect(dbname=creds_mm["db"],user=creds_mm["u"],host=creds_mm["h"],password=creds_mm["pw"]) as conn_mm:
with ps.connect(dbname=creds_chDB["db"],user=creds_chDB["u"],host=creds_chDB["h"],password=creds_chDB["pw"]) as conn_chDB:
    aggregator = "COUNT(DISTINCT user_id)"
    for hashtag in hashtags:
        #count_osmStats = osmStatsQuery(timerange,hashtag,conn=conn_mm,aggregator=aggregator)[0][0]
        count_chDB = changesetDbQuery(timerange,hashtag,conn=conn_chDB,aggregator=aggregator)[0][0]

        print("#####################")
        #print(f"hashtag: {hashtag}")
        print(f"measurment: Number of (unique) users")
        #print(f"Result OSM-stats API: {count_osmStats}")
        print(f"Result OSM changeset DB: {count_chDB}")
        print(f"Result Parquet file: toDo")

UndefinedFunction: function lower(boolean) does not exist
LINE 3:                 WHERE (LOWER(tags ? 'comment') LIKE '%missin...
                               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


## EQ2 Number of OSM Changesets

This is the number of distinct OSM changesets contributed to a particular hashtag since the start of collecting data. This number is of less important for partners or reporting at mapathons and often this causes confusion.

However, internally this number will be helpful to compare different processing approaches.

In [ ]:
with ps.connect(dbname=creds_mm["db"],user=creds_mm["u"],host=creds_mm["h"],password=creds_mm["pw"]) as conn_mm:
    with ps.connect(dbname=creds_chDB["db"],user=creds_chDB["u"],host=creds_chDB["h"],password=creds_chDB["pw"]) as conn_chDB:
        aggregator = "COUNT(DISTINCT id)"
        for hashtag in hashtags:
            count_osmStats = osmStatsQuery(timerange,hashtag,conn=conn_mm,aggregator=aggregator)[0][0]
            count_chDB = changesetDbQuery(timerange,hashtag,conn=conn_mm,aggregator=aggregator)[0][0]
            
            print("#####################")
            print(f"hashtag: {hashtag}")
            print(f"measurment: Number of OSM Changesets")
            print(f"Result OSM-stats API: {count_osmStats}")
            print(f"Result OSM changeset DB: {count_chDB}")
            print(f"Result Parquet file: toDo")
            

## EQ3 Number of Total Map Edits


In [ ]:
with ps.connect(dbname=creds_mm["db"],user=creds_mm["u"],host=creds_mm["h"],password=creds_mm["pw"]) as conn_mm:
    with ps.connect(dbname=creds_chDB["db"],user=creds_chDB["u"],host=creds_chDB["h"],password=creds_chDB["pw"]) as conn_chDB:
        aggregator = "SUM(num_changes)"
        for hashtag in hashtags:
            count_osmStats = osmStatsQuery(timerange,hashtag,conn=conn_mm,aggregator=aggregator)[0][0]
            count_chDB = changesetDbQuery(timerange,hashtag,conn=conn_mm,aggregator=aggregator)[0][0]
            
            print("#####################")
            print(f"hashtag: {hashtag}")
            print(f"measurment: Number of OSM Changesets")
            print(f"Result OSM-stats API: {count_osmStats}")
            print(f"Result OSM changeset DB: {count_chDB}")
            print(f"Result Parquet file: toDo")